In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import pickle
import copy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [3]:
wrs = pickle.load(open("../data/rbs_senior.pkl","rb"))

In [6]:
value_inputs = wrs[["Rush","Yds_x","TD_x","Fmb","G_x"]]


value_inputs["Value"] = (value_inputs["Yds_x"] + 50 * (value_inputs["TD_x"] - 0.5 * value_inputs["Fmb"])) / (0.5 * value_inputs["G_x"])
y = value_inputs["Value"]
X = wrs.loc[:,"Att":"Conf"]

X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.8, random_state=29)
data = X_train.merge(pd.DataFrame(y_train), left_index = True, right_index = True)
data = data.dropna()

/home/leaferickson/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [7]:
lm = smf.ols('Value ~ Yds_y + TD_y + Att + Avg + Year + Conf', data = data)
most_naive_model = lm.fit()
most_naive_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Value   R-squared:                       0.149
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     17.23
Date:                Fri, 20 Jul 2018   Prob (F-statistic):           2.07e-18
Time:                        00:54:12   Log-Likelihood:                -3162.6
No. Observations:                 597   AIC:                             6339.
Df Residuals:                     590   BIC:                             6370.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      224.7749    313.163      0.718      0.473    -390.275     839.824
Conf[T.True]    16.7451      6.124      2.734      0.006       4.718      28.772
Yds_y            0.0456      0.020      2.263      0.024       0.006       0.085
TD_y             2.4276      0.565      4.297      0.000       1.318       3.537
Att             -0.2072      0.105     -1.968      0.049      -0.414      -0.000
Avg             -0.1947      2.865     -0.068      0.946      -5.821       5.432
Year            -0.1018      0.158     -0.644      0.520      -0.412       0.209
==============================================================================
Omnibus:                       94.432   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              144.116
Skew:                           1.039   Prob(JB):                     5.08e-32
Kurtosis:                       4.215   Cond. No.                     3.48e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.48e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
lm = smf.ols('Value ~ Yds_y + TD_y + Att + Year + Conf', data = data)
most_naive_model = lm.fit()
most_naive_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Value   R-squared:                       0.149
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     20.72
Date:                Fri, 20 Jul 2018   Prob (F-statistic):           4.30e-19
Time:                        00:54:40   Log-Likelihood:                -3162.6
No. Observations:                 597   AIC:                             6337.
Df Residuals:                     591   BIC:                             6364.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      223.9866    312.684      0.716      0.474    -390.121     838.094
Conf[T.True]    16.7348      6.117      2.736      0.006       4.722      28.748
Yds_y            0.0446      0.013      3.383      0.001       0.019       0.070
TD_y             2.4284      0.564      4.303      0.000       1.320       3.537
Att             -0.2019      0.070     -2.873      0.004      -0.340      -0.064
Year            -0.1019      0.158     -0.645      0.519      -0.412       0.208
==============================================================================
Omnibus:                       94.376   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              143.987
Skew:                           1.039   Prob(JB):                     5.41e-32
Kurtosis:                       4.214   Cond. No.                     3.48e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.48e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
lm = smf.ols('Value ~ Yds_y + TD_y + Att + Conf', data = data)
most_naive_model = lm.fit()
most_naive_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Value   R-squared:                       0.149
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     25.82
Date:                Fri, 20 Jul 2018   Prob (F-statistic):           9.63e-20
Time:                        00:54:55   Log-Likelihood:                -3162.8
No. Observations:                 597   AIC:                             6336.
Df Residuals:                     592   BIC:                             6358.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       22.2158      7.468      2.975      0.003       7.548      36.883
Conf[T.True]    16.7591      6.114      2.741      0.006       4.752      28.766
Yds_y            0.0437      0.013      3.336      0.001       0.018       0.069
TD_y             2.3780      0.559      4.257      0.000       1.281       3.475
Att             -0.1995      0.070     -2.844      0.005      -0.337      -0.062
==============================================================================
Omnibus:                       92.823   Durbin-Watson:                   1.939
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              140.051
Skew:                           1.031   Prob(JB):                     3.87e-31
Kurtosis:                       4.173   Cond. No.                     4.90e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.9e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print(mean_squared_error(y_train,lr.predict(X_train)))
print(mean_squared_error(y_test,lr.predict(X_test)))

2338.391014799507
1776.2170171959851


In [11]:
lr = Lasso()
lr.fit(X_train, y_train)
print(mean_squared_error(y_train,lr.predict(X_train)))
print(mean_squared_error(y_test,lr.predict(X_test)))

2348.08359585325
1752.9900585057712


In [12]:
lr = Ridge()
lr.fit(X_train, y_train)
print(mean_squared_error(y_train,lr.predict(X_train)))
print(mean_squared_error(y_test,lr.predict(X_test)))

2338.398245739542
1775.3678433915554


In [13]:
lr = ElasticNet()
lr.fit(X_train, y_train)
print(mean_squared_error(y_train,lr.predict(X_train)))
print(mean_squared_error(y_test,lr.predict(X_test)))

2361.338266100455
1746.3851027818655


In [15]:
X2 = X_train[["Yds_y", "Att", "Conf", "TD_y", "Year"]]
X2["int"] = 1
X2["Year_Att"] = np.dot(X2["Year"], X2["Att"])
lr = Lasso()
lr.fit(X2, y_train)
print(mean_squared_error(y_train,lr.predict(X2)))
X3 = X_test[["Yds_y", "Att", "Conf", "TD_y", "Year"]]
X3["int"] = 1
X3["Year_Att"] = np.dot(X3["Year"], X3["Att"])
print(mean_squared_error(y_test,lr.predict(X3)))

2348.083533991831
1752.9920380062954


In [16]:
X2 = X_train[["Yds_y", "Att", "Conf", "TD_y", "Year"]]
X2["int"] = 1
lr = Lasso()
lr.fit(X2, y_train)
print(mean_squared_error(y_train,lr.predict(X2)))
X3 = X_test[["Yds_y", "Att", "Conf", "TD_y", "Year"]]
X3["int"] = 1
print(mean_squared_error(y_test,lr.predict(X3)))

2348.083533991831
1752.9920380062954


In [23]:
X2 = X_train[["Yds_y", "Att", "TD_y", "Year"]]
X2["int"] = 1
X2["Year_Att"] = (X2["Year"] * X2["Att"])
lr = ElasticNet()
lr.fit(X2, y_train)
print(mean_squared_error(y_train,lr.predict(X2)))
X3 = X_test[["Yds_y", "Att", "TD_y", "Year"]]
X3["int"] = 1
X3["Year_Att"] = (X3["Year"] * X3["Att"])
print(mean_squared_error(y_test,lr.predict(X3)))

2367.145372696814
1742.7365429074403


/home/leaferickson/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/leaferickson/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/leaferickson/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-cop